# 1/ Env Setup
Load necessary libraries to run this notebook. <br>
All libraries are cited in ```requirements.txt```. <br>
Documentation: https://docs.pytorch.org/vision/main/models/generated/torchvision.models.detection.retinanet_resnet50_fpn_v2.html

## 1.1/ Import dependencies
Load libraries:

In [38]:
import sys  
import os

current_dir = os.getcwd() # path to the current working directory (notebook location)
project_root = os.path.abspath(os.path.join(current_dir, '..')) # path to project root

if project_root not in sys.path: # add project root to sys.path
    sys.path.insert(0, project_root)
print(f"Project root added to sys.path: {project_root}")

Project root added to sys.path: /Users/litani/Documents/myCode/steel-defects


In [39]:
from pathlib import Path
import torch 
import torchvision
from torchvision.models.detection import retinanet_resnet50_fpn_v2
from torchvision.models.detection.retinanet import RetinaNetClassificationHead
import albumentations as A # note that albumentationsx was installed and but you still albumentations
from albumentations.pytorch import ToTensorV2
import numpy as np
from src.utils.parse_xml import parse_xml

## 1.2/ Set reproducibility
Device and seed:

In [40]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
torch.manual_seed(42)
np.random.seed(42)

# 2/ Configuration Management
Define:
- image path
- model hyperparameters
- hardware


In [41]:
class Config:
    # Paths
    DATA_ROOT = Path("../data/raw")
    TRAIN_IMG = DATA_ROOT / "train_images"
    TRAIN_ANN = DATA_ROOT / "train_annotations"
    VAL_IMG = DATA_ROOT / "valid_images"
    VAL_ANN = DATA_ROOT / "valid_annotations"

    # Model parameters
    NUM_CLASSES = 7 # 6 defects + 1 background
    BACKBONE_PRETRAINED = True 
    
    # Training hyperparameters
    BATCH_SIZE = 16  # no mention of batch size in the paper
    NUM_EPOCHS = 5 # 24 epochs based on paper. Reduced for quicker testing
    LEARNING_RATE = 0.0025 # 0.0025 based on paper
    MOMENTUM = 0.9 # 0.9 based on paper

    # Hardware
    DEVICE = device
    NUM_WORKERS = 8
    PIN_MEMORY = True if torch.cuda.is_available() else False

config = Config()

In [42]:
config

# 3/ Dataset Class
- Load images and annotations into PyTorch format. 
- This is necessary since RetineNet excepts a dictionary format. The latter requires XML parsing.

In [43]:
class SteelDefectDataset(torch.utils.data.Dataset): # inherits from PyTorch Dataaset class, making it compatible w/ DataLoader for batch and // processing
    def __init__(self, img_dir, ann_dir, transforms = None): # takes 3 inputs: image directory, annotation directory, and optional transforms (default no augmentation)
        self.img_dir = Path(img_dir)
        self.ann_dir = Path(ann_dir)
        self.transforms = transforms

        # Logic of loading a dataset
        self.images = sorted(list(self.img_dir.glob("*.jpg"))) # find all .jpg files, sort them in order
        self.class_map = { 
            "crazing" : 1,
            "inclusion" : 2,
            "patches" : 3,
            "pitted_surface" : 4,
            "rolled-in_scale" : 5,
            "scratches" : 6,
        }                                                  # mapping defect names (catagorical) to integers (numerical)
    def __len__(self): # runs total number of images
        return len(self.images) 
    
    def __getitem__(self, idx):
        # Load image
        img_path = self.images[idx]
        image = cv2.imread(str(img_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # convert BGR, i.e. OpenCV format, to RGB, i.e. PyTorch/RetinaNet format

        # Parse annotations using parse_xml function
        xml_path = self.ann_dir / img_path.name.replace(".jpg", ".xml")
        boxes_data = parse_xml(str(xml_path))

        # Convert XML to lists: [x1, y1, x2, y2], labels = integers
        boxes = []
        labels = []
        for box in boxes_data:
            boxes.append([box["xmin"], box["ymin"], box["xmax"], box["ymax"]])
            labels.append(self.class_map[box["label"]])

        target = {
            "boxes": torch.as_tensor(boxes, dtype = torch.float32),
            "labels": torch.as_tensor(labels, dtype = torch.int64),
            "image_id": torch.as_tensor([idx])
        } # lists are packages into dictionary to be feed to RetinaNet, becoming PyTorch tensors.

        # Image augmetations: returns a tuple (image tensor, target dictionary)
        if self.transforms:
            transformed = self.transforms(image = image, bboxes = target["boxes"], labels = target["labels"])
            image = transformed["image"]
            target["boxes"] = torch.as_tensor(transformed["bboxes"], dtype = torch.float32)
        
        return image, target

# 4/ Data Augmentation
- We have 1800 images, resorting to image augmentation is mandatory to avoid overfitting. 
- Geometric transformation, simplist form, will be applied as a quick fix:
    - Horizental/Vertical flips
    - Rotate by 90
    - Others: brightness, contrast, adding random noise
- **NB:** OpenCV stores images as [Height in pixels, Width in pixels, RGB] while PyTorch expects [channel, height, width]

In [44]:
def get_train_transforms():
    return A.Compose([
        A.HorizontalFlip(p = .5), # probability of 50% to flip the image left-right >> defects visible from any orientation
        A.VerticalFlip(p = .5), # flip top-bottom >> surfaces viewed from any angle
        A.RandomRotate90(p = .5), # rotate 90, 180 or 270 degrees >> increase orientation variety
        A.RandomBrightnessContrast(p = .3), # adjust brightness/contrast >> image quality
        A.GaussNoise(p = .2), # add random noise
        ToTensorV2() # converts NumPy array to PyTorch tensor, divides pix by 255 ([0,1] range), permutes format from [H,W,C] to [C,H,W].
    ], bbox_params = A.BboxParams(format = "pascal_voc", label_fields = ["labels"])) # XML annotation has a Pascal Voc format

def get_val_transforms(): # for validation on clean images, converting only to tensor format, no augmentation.
    return A.Compose([
        ToTensorV2()
    ], bbox_params = A.BboxParams(format = "pascal_voc", label_fields = ["labels"]))

# 5/ Model Initilization
- Apply transfer learning where pretrained RetineNet is loaded the changes are applied based on the dataset

In [ ]:
def create_model(num_classes, pretrained = True):
    # Load pretrained RetineNet w/ ResNet50 backbone,
    model = retinanet_resnet50_fpn_v2(wights = "DEFAULT" if pretrained else None)  # DEFAULT loads ImageNet pretrained weights for transfer learning
    
    # Replace head so that model learns defect-specific patterns
    num_anchors = model.head.classification_head.num_anchors # default is 9 anchors per location >> 3 scales x 3 aspect ratios
    model.head.classification_head = RetinaNetClassificationHead(
        in_channels = 256,          # Input: 256 features from FPN   
        num_anchors = num_anchors,  # Process: 9 anchors per location
        num_classes = num_classes   # Output: 7 classes scores per anchor
    )
    return model

model = create_model(config.NUM_CLASSES).to(device) # Create model instance and move to device CPU/GPU, config.NUM_CLASSES = 7 includes background

# 6/ Data Loaders
- collate_func is a fucntion that works on the collation process of RetinaNet since images have variable bbox counts.

In [ ]:
# Create train dataset with augmentations and val dataset without augmentations, only format conversion
train_dataset = SteelDefectDataset(
    config.TRAIN_IMG,
    config.TRAIN_ANN,
    transforms = get_train_transforms()
)

val_dataset = SteelDefectDataset(
    config.VAL_IMG,
    config.VAL_ANN,
    transforms = get_val_transforms()
)

# Create data loaders
def collate_func(batch):
    return tuple(zip(*batch)) # transpose batch to a list of tensors instead of stack of images into a single tensor 
                                #[(img1, target1), (img2, target2)] >> ([img1, img2],[target1, target2])

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size = config.BATCH_SIZE,
    shuffle = True, # shuffle training data for better generalization
    num_workers = config.NUM_WORKERS,
    pin_memory = config.PIN_MEMORY, # only useful if using GPU
    collate_fn = collate_func 
)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size = config.BATCH_SIZE,
    shuffle = False, # validation data should be consistent
    num_workers = config.NUM_WORKERS,
    pin_memory = config.PIN_MEMORY,
    collate_fn = collate_func
)